# Gas

> Fill in a module description here

In [ ]:
# | default_exp gas.core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

In [ ]:
# | export
from exex.basics import *

### Properties


In [ ]:
# | export
class IdealGasConstant(PropertyObservable):
    def __init__(self, compound):
        super().__init__(compound)
        self.abbrv = "R"

In [ ]:
# | export
class IsIdealGas(PropertyObservable):
    def __init__(self, compound):
        super().__init__(compound)
        self.abbrv = "is_ideal_gas"
        self.unit = None

    def __bool__(self):
        return False

#### Mixture of Gases and Partial Pressure

The number of mole of a component in a mixture is called mole fraction.
$$\chi_{\mathrm{a}}=\frac{n_{\mathrm{a}}}{n_{\mathrm{total}}}$$

In [ ]:
# | export
class MoleFraction(PropertyObservable):
    def __init__(self, cmp):
        super().__init__(cmp)
        pass

In [ ]:
# | export
class PartialPressure(PropertyObservable):
    def __init__(self, compound):
        super().__init__(compound)

### Laws

- mass mole ratio
- mole volume ratio

##### The Simple Gas Law

In [ ]:
# | export
class BoyleLaw(Law):
    def __init__(self, compound: Compound):
        super().__init__()
        self.compound = compound
        # self.properties = [Pressure, Volume]
        self.properties = [{"object": Pressure}, {"object": Volume}]

    def expr(self):
        pass

In [ ]:
# | export
class CharlesLaw(Law):
    def __init__(self, compound: Compound) -> None:
        super().__init__()
        self.compound = compound
        # self.properties = [Volume, Temperature]
        self.properties = [{"object": Volume}, {"object": Temperature}]

    def expr(self):
        pass

In [ ]:
# | export
class AvogadroLaw(Law):
    def __init__(self, compound: Compound) -> None:
        super().__init__()
        self.compound = compound
        # self.properties = [Volume, Mole]
        self.properties = [{"object": Volume}, {"object": Mole}]

    def expr(self):
        pass

##### The Ideal Gas Law

In [ ]:
pressure = {
    "pressure": {"unit": "pascal", "property": Pressure},
    "x": {"unit": "pascal", "property": Pressure},
}

In [ ]:
# | export
class IdealGasLaw(Law):
    def __init__(self, compound: Compound) -> None:
        # super().__init__()
        self.compound = compound
        # self.properties = [Pressure, Volume, Mole, Temperature, IsIdealGas]
        self.properties = [
            {"object": Pressure, "unit": "atm"},
            {"object": Volume},
            {"object": Mole},
            {"object": Temperature},
            {"object": IdealGasConstant},
            {"object": IsIdealGas},
        ]

    def expr(self, t, **kwargs):

        cmp = self.compound
        params = {"t": t}

        left_side = cmp.get_prop("pressure", *params) * cmp.get_prop("volume", *params)
        right_side = (
            cmp.get_prop("mole", *params)
            * cmp.get_prop("ideal_gas_constant", *params)
            * cmp.get_prop("temperature", *params)
        )

        return smp.Eq(left_side, right_side)

### States

#### Ideal Gas State

Return `True` if the ideal gas equation holds

In [ ]:
# | export
from abc import ABC, abstractmethod

In [ ]:
# | export
class State(ABC):
    def __init__(self, context):
        self.context = context

    @abstractmethod
    def __bool__(self, timestep):
        pass

### Gas Compound

Gas -> `self._config_laws` -> `init law` -> `law._config_properties`

In [ ]:
# | export
class Gas(Compound):
    def __init__(self, formula: str) -> None:  # the chemical formula
        super().__init__(formula)

        # self._laws = [BoyleLaw, CharlesLaw, AvogadroLaw, IdealGasLaw]
        # self._config_laws()

        self.add_laws = [BoyleLaw, CharlesLaw, AvogadroLaw, IdealGasLaw, MassMoleRatio]

Sarin gas is developed by Nazi during WWII

In [ ]:
C4H10FO2P = Gas("C4H10FO2P")

NameError: name 'MassMoleRatio' is not defined

In [ ]:
C4H10FO2P

In [ ]:
C4H10FO2P.properties

In [ ]:
C4H10FO2P.laws["charles_law"]

NameError: name 'C4H10FO2P' is not defined

In [ ]:
C4H10FO2P.laws["ideal_gas_law"](t=2)

NameError: name 'C4H10FO2P' is not defined

In [ ]:
# | hide
# test laws
test_eq(
    L(C4H10FO2P.laws),
    ["mass_mole_ratio", "boyle_law", "charles_law", "avogadro_law", "ideal_gas_law"],
)

In [ ]:
# | hide
# test a properties
test_eq(C4H10FO2P.properties["mass"].abbrv, "m")
test_eq_type(type(C4H10FO2P.get_prop("mass", t=2)), smp.core.symbol.Symbol)

In [ ]:
# | hide
# test a list of properties
test_eq(
    L(C4H10FO2P.properties),
    [
        "mass",
        "mole",
        "molar_mass",
        "pressure",
        "volume",
        "temperature",
        "ideal_gas_constant",
        "is_ideal_gas",
    ],
)